In [1]:
import pandas as pd

In [2]:
data = [[1, 1, None, 1],
        [2, 2, 3, 2],
        [3, 3, 3, 3],
        [3, 3, 3, 3],
        [2, 2, 2, 2],
        [1, 2, 3, 4],
        [4, 4, 4, 4],
        [1, 1, 2, 1],
        [2, 2, 2, 2],
        [None, 5, 5, 5],
        [None, None, 1, 1],
        [None, None, 3, None]]

In [4]:
df = pd.DataFrame(data, columns=["A", "B", "C", "D"])
df

,A,B,C,D
0,1.0,1.0,NaN,1.0
1,2.0,2.0,3.0,2.0
2,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0
4,2.0,2.0,2.0,2.0
5,1.0,2.0,3.0,4.0
6,4.0,4.0,4.0,4.0
7,1.0,1.0,2.0,1.0
8,2.0,2.0,2.0,2.0
9,NaN,5.0,5.0,5.0


In [5]:
values = [1, 2, 3, 4, 5]

In [9]:
counts = pd.DataFrame()
for value in values:
    counts[value] = (df == value).astype(int).sum(axis=1)
counts

,1,2,3,4,5
0,3,0,0,0,0
1,0,3,1,0,0
2,0,0,4,0,0
3,0,0,4,0,0
4,0,4,0,0,0
5,1,1,1,1,0
6,0,0,0,4,0
7,3,1,0,0,0
8,0,4,0,0,0
9,0,0,0,0,3
